```
--- Day 5: Supply Stacks ---
The expedition can depart as soon as the final supplies have been unloaded from the ships. Supplies are stored in stacks of marked crates, but because the needed supplies are buried under many other crates, the crates need to be rearranged.

The ship has a giant cargo crane capable of moving crates between stacks. To ensure none of the crates get crushed or fall over, the crane operator will rearrange them in a series of carefully-planned steps. After the crates are rearranged, the desired crates will be at the top of each stack.

The Elves don't want to interrupt the crane operator during this delicate procedure, but they forgot to ask her which crate will end up where, and they want to be ready to unload them as soon as possible so they can embark.

They do, however, have a drawing of the starting stacks of crates and the rearrangement procedure (your puzzle input). For example:

    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 

move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2
In this example, there are three stacks of crates. Stack 1 contains two crates: crate Z is on the bottom, and crate N is on top. Stack 2 contains three crates; from bottom to top, they are crates M, C, and D. Finally, stack 3 contains a single crate, P.

Then, the rearrangement procedure is given. In each step of the procedure, a quantity of crates is moved from one stack to a different stack. In the first step of the above rearrangement procedure, one crate is moved from stack 2 to stack 1, resulting in this configuration:

[D]        
[N] [C]    
[Z] [M] [P]
 1   2   3 
In the second step, three crates are moved from stack 1 to stack 3. Crates are moved one at a time, so the first crate to be moved (D) ends up below the second and third crates:

        [Z]
        [N]
    [C] [D]
    [M] [P]
 1   2   3
Then, both crates are moved from stack 2 to stack 1. Again, because crates are moved one at a time, crate C ends up below crate M:

        [Z]
        [N]
[M]     [D]
[C]     [P]
 1   2   3
Finally, one crate is moved from stack 1 to stack 2:

        [Z]
        [N]
        [D]
[C] [M] [P]
 1   2   3
The Elves just need to know which crate will end up on top of each stack; in this example, the top crates are C in stack 1, M in stack 2, and Z in stack 3, so you should combine these together and give the Elves the message CMZ.

After the rearrangement procedure completes, what crate ends up on top of each stack?
```

In [13]:
#r "nuget: FParsec"
open FParsec

let input = System.IO.File.ReadAllLines "./input.txt"

let (initialStateInput, instructionsInput) = 
    input |> Array.splitAt (Array.findIndex (fun line -> line = "") input)

type State = Map<int, char array>
type Instruction =
    {
        Amount: int;
        From: int;
        To: int;
    }

let initialState = 
    initialStateInput
        |> Array.map (Seq.map (fun c -> if List.contains c ['[';']'] then ' ' else c) >> Array.ofSeq)
        |> Array.transpose
        |> Array.map (Array.filter ((=) ' ' >> not))
        |> Array.filter (Array.isEmpty >> not)
        |> Array.map (fun arr -> Array.splitAt (Array.length arr - 1) arr)
        |> Array.fold (fun state (value, key) -> Map.add (Array.head key |> Char.ToString |> Int32.Parse ) value state) Map.empty

let instructionParser: Parser<Instruction, unit> =
    pipe3 
        (pstring "move " >>. pint32) 
        (pstring " from " >>. pint32) 
        (pstring " to " >>. pint32) 
        (fun amount from target -> {Amount = amount; From = from; To = target})

let instructions =
    instructionsInput
        |> Array.filter (String.IsNullOrEmpty >> not)
        |> Array.map (run instructionParser >> function | Success (a, _, _ ) -> a | Failure _ -> failwith "Parsing error")

let runInstruction (state: State) (instruction: Instruction) : State =
    let stack = Array.take instruction.Amount state.[instruction.From]
    state
        |> Map.change instruction.From (Option.map (Array.skip instruction.Amount))
        |> Map.change instruction.To (Option.map (fun a -> Array.concat [(Array.rev stack); a]))

let finalState = Array.fold (runInstruction) initialState instructions 

let getResult (state: State) : string  =
    Map.values state
    |> Seq.map (Seq.head)
    |> Array.ofSeq
    |> System.String

getResult finalState

Installed Packages FParsec, 1.1.1

SPFMVDTZT

In [14]:
let runInstruction2 (state: State) (instruction: Instruction) : State =
    let stack = Array.take instruction.Amount state.[instruction.From]
    state
        |> Map.change instruction.From (Option.map (Array.skip instruction.Amount))
        |> Map.change instruction.To (Option.map (fun a -> Array.concat [ stack; a]))

getResult (Array.fold runInstruction2 initialState instructions)

ZFSJBPRFP